## Загрузка и чтение данных

In [2]:
!pip install catboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score

In [4]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [5]:
df = pd.read_csv("/content/drive/MyDrive/weatherAUS.csv")

In [6]:
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,...,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No
145456,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,...,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No
145457,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,...,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No
145458,2017-06-24,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,...,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No,No


### Сводка данных о датасете

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

## Произведено кодирование категориальных признаков

In [8]:
label_encoder = LabelEncoder()

In [9]:
df['Date'] = pd.to_datetime(df['Date'])

In [10]:
df['Location'] = label_encoder.fit_transform(df['Location'])
df['WindGustDir'] = label_encoder.fit_transform(df['WindGustDir'].astype(str))
df['WindDir9am'] = label_encoder.fit_transform(df['WindDir9am'].astype(str))
df['WindDir3pm'] = label_encoder.fit_transform(df['WindDir3pm'].astype(str))
df['RainToday'] = label_encoder.fit_transform(df['RainToday'].astype(str))
df['RainTomorrow'] = label_encoder.fit_transform(df['RainTomorrow'].astype(str))

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           145460 non-null  datetime64[ns]
 1   Location       145460 non-null  int64         
 2   MinTemp        143975 non-null  float64       
 3   MaxTemp        144199 non-null  float64       
 4   Rainfall       142199 non-null  float64       
 5   Evaporation    82670 non-null   float64       
 6   Sunshine       75625 non-null   float64       
 7   WindGustDir    145460 non-null  int64         
 8   WindGustSpeed  135197 non-null  float64       
 9   WindDir9am     145460 non-null  int64         
 10  WindDir3pm     145460 non-null  int64         
 11  WindSpeed9am   143693 non-null  float64       
 12  WindSpeed3pm   142398 non-null  float64       
 13  Humidity9am    142806 non-null  float64       
 14  Humidity3pm    140953 non-null  float64       
 15  

### Можно установить что в наборе данных имеются пустые значения, произведена замена пустых значений на средние

In [12]:
df.fillna(df.mean(), inplace=True)

In [13]:
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,2,13.4,22.900000,0.6,5.468232,7.611178,13,44.00000,13,...,71.0,22.0,1007.7,1007.1,8.000000,4.50993,16.9,21.8,0,0
1,2008-12-02,2,7.4,25.100000,0.0,5.468232,7.611178,14,44.00000,6,...,44.0,25.0,1010.6,1007.8,4.447461,4.50993,17.2,24.3,0,0
2,2008-12-03,2,12.9,25.700000,0.0,5.468232,7.611178,15,46.00000,13,...,38.0,30.0,1007.6,1008.7,4.447461,2.00000,21.0,23.2,0,0
3,2008-12-04,2,9.2,28.000000,0.0,5.468232,7.611178,4,24.00000,9,...,45.0,16.0,1017.6,1012.8,4.447461,4.50993,18.1,26.5,0,0
4,2008-12-05,2,17.5,32.300000,1.0,5.468232,7.611178,13,41.00000,1,...,82.0,33.0,1010.8,1006.0,7.000000,8.00000,17.8,29.7,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,41,2.8,23.400000,0.0,5.468232,7.611178,0,31.00000,9,...,51.0,24.0,1024.6,1020.3,4.447461,4.50993,10.1,22.4,0,0
145456,2017-06-22,41,3.6,25.300000,0.0,5.468232,7.611178,6,22.00000,9,...,56.0,21.0,1023.5,1019.1,4.447461,4.50993,10.9,24.5,0,0
145457,2017-06-23,41,5.4,26.900000,0.0,5.468232,7.611178,3,37.00000,9,...,53.0,24.0,1021.0,1016.8,4.447461,4.50993,12.5,26.1,0,0
145458,2017-06-24,41,7.8,27.000000,0.0,5.468232,7.611178,9,28.00000,10,...,51.0,24.0,1019.4,1016.5,3.000000,2.00000,15.1,26.0,0,0


### Разделение данных на признаки и целевую переменную

In [14]:
X = df.drop(columns=['RainTomorrow', 'Date'])
y = df['RainTomorrow']

## Метод CatBoost

CatBoost - это высокоэффективная библиотека градиентного бустинга, разработанная компанией Яндекс. Она специально оптимизирована для работы с категориальными признаками и автоматически обрабатывает их без необходимости предварительного кодирования.

Гиперпараметры: iterations: количество итераций обучения; learning_rate: скорость обучения; depth - максимальная глубина дерева; border_count: количество корзин для числовых признаков. Этот параметр влияет на скорость обучения и качество модели.

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
catboost_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, border_count=32)
catboost_cv_scores = cross_val_score(catboost_classifier, X_train, y_train, cv=5)

0:	learn: 0.9868155	total: 105ms	remaining: 1m 44s
1:	learn: 0.8994384	total: 161ms	remaining: 1m 20s
2:	learn: 0.8308184	total: 211ms	remaining: 1m 9s
3:	learn: 0.7738326	total: 266ms	remaining: 1m 6s
4:	learn: 0.7280553	total: 322ms	remaining: 1m 4s
5:	learn: 0.6878259	total: 371ms	remaining: 1m 1s
6:	learn: 0.6543525	total: 423ms	remaining: 1m
7:	learn: 0.6262831	total: 471ms	remaining: 58.4s
8:	learn: 0.6017769	total: 532ms	remaining: 58.6s
9:	learn: 0.5799840	total: 581ms	remaining: 57.5s
10:	learn: 0.5612221	total: 635ms	remaining: 57.1s
11:	learn: 0.5442103	total: 691ms	remaining: 56.9s
12:	learn: 0.5302821	total: 750ms	remaining: 57s
13:	learn: 0.5175044	total: 803ms	remaining: 56.6s
14:	learn: 0.5066935	total: 863ms	remaining: 56.6s
15:	learn: 0.4970721	total: 925ms	remaining: 56.9s
16:	learn: 0.4885927	total: 993ms	remaining: 57.4s
17:	learn: 0.4803604	total: 1.04s	remaining: 57s
18:	learn: 0.4731357	total: 1.09s	remaining: 56.5s
19:	learn: 0.4666347	total: 1.14s	remaining: 5

In [17]:
print("CatBoost CV Scores:", catboost_cv_scores)
print("CatBoost Mean CV Score:", catboost_cv_scores.mean())

CatBoost CV Scores: [0.8548595  0.84708258 0.85069176 0.85184549 0.85270485]
CatBoost Mean CV Score: 0.8514368355275183
